In [1]:
import sys, os, io, json, numpy as np, random
#from sklearn.metrics import roc_auc_score
#import xgboost as xgb
import tensorflow as tf
import pandas as pd

/usr/local/Cellar/python/3.6.4_3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/local/Cellar/python/3.6.4_3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
pool = np.load('../prepare_dataset/pool_00.npz')
features_bool, f_time, label = (pool[name] for name in ['features', 'f_time', 'label'])

In [3]:
features = np.random.normal(loc=features_bool*1.0)

In [200]:
def create_graph():
    graph = tf.Graph()
    with graph.as_default():
        features = tf.placeholder(dtype=tf.float64)
        label = tf.placeholder(dtype=tf.float64)
        bias = tf.placeholder(dtype=tf.float64)
        ax = tf.transpose(tf.nn.top_k(-tf.transpose(features), k = tf.shape(features)[-2]).indices)
        y = tf.gather(label, ax)[:, :, 0]
        b = tf.gather(bias, ax)[:, :, 0]
        #T = tf.gather(features, ax, axis=0)
        T = tf.contrib.framework.sort(features, axis=0)
        l_curr = -(y * tf.log_sigmoid(b) + (1 - y) * tf.log_sigmoid(-b))
        l_der1 = -y * tf.sigmoid(-b) + (1 - y) * tf.sigmoid(b)
        l_der2 = tf.sigmoid(-b) * tf.sigmoid(b)
        cum_l = tf.cumsum(l_curr)[:-1, :]  # -1
        cum_l_der1 = tf.cumsum(l_der1)[:-1, :]  # -1
        cum_l_der2 = tf.cumsum(l_der2)[:-1, :]  # -1
        rev_cum_l = tf.cumsum(l_curr, reverse=True)[1:, :]  # -1
        rev_cum_l_der1 = tf.cumsum(l_der1, reverse=True)[1:, :] # -1
        rev_cum_l_der2 = tf.cumsum(l_der2, reverse=True)[1:, :] # -1
        delta_up = -cum_l_der1 / cum_l_der2  # -1
        delta_down = -rev_cum_l_der1 / rev_cum_l_der2 # -1
        loss_up = cum_l + 0.5 * delta_up * cum_l_der1
        loss_down = rev_cum_l + 0.5 * delta_down * rev_cum_l_der1
        whole_loss = (loss_up + loss_down)/tf.to_double(tf.shape(features)[0])
        #######
        #-----#
        #-----#   -->   iiiIiii (best_loss_argmin0), mmmMmmm (min_loss_axis0)
        #-----#
        #######
        best_loss_argmin0 = tf.argmin(whole_loss, axis=0)
        min_loss_axis0 = tf.reduce_min(whole_loss, axis=0)
        # mmmMmmm  ->    I (best_loss_argmin1), M (best_loss)
        best_loss_argmin1 = tf.argmin(min_loss_axis0, axis=0)
        best_loss = tf.reduce_min(min_loss_axis0, axis=0)
        # iiiIiii  ->  I
        best_index = best_loss_argmin0[best_loss_argmin1]
        thr = (T[best_index, best_loss_argmin1] + T[best_index+1, best_loss_argmin1])/2
        
    return {'graph': graph,
            'features': features,
            'bias': bias, 
            'label': label,
            'l_curr': l_curr,
            'l_der1': l_der1,
            'l_der2': l_der2,
            'cum_l_der1': cum_l_der1,
            'cum_l_der2': cum_l_der2,
            'rev_cum_l_der1': rev_cum_l_der1,
            'rev_cum_l_der2': rev_cum_l_der2,
            'delta_up': delta_up, 
            'delta_down': delta_down,
            'cum_l': cum_l,
            'rev_cum_l': rev_cum_l,
            'whole_loss': whole_loss,
            'best_loss_argmin0' : best_loss_argmin0,
            'min_loss_axis0': min_loss_axis0,
            'best_loss_argmin1': best_loss_argmin1,
            'best_loss': best_loss,
            'best_index': best_index,
            'T': T,
            'thr': thr,
            'y': y
           }

In [201]:
ret = create_graph()
graph, features_t, bias_t, label_t, y_t, T_t = (ret[t] for t in ['graph', 'features', 'bias', 'label', 'y', 'T'])
with tf.Session(graph=graph) as s:
    (y_v, T_v, l_curr_v, whole_loss_v, cum_l_v, rev_cum_l_v,
       thr_v, bloss_v, bind_v, test_val, 
       test_val2) = s.run([y_t, T_t, 
                           ret['l_curr'], ret['whole_loss'], ret['cum_l'], ret['rev_cum_l'],
                           ret['thr'], ret['best_loss'], ret['best_index'],
                           ret['best_loss_argmin0'], ret['min_loss_axis0']], 
                          {features_t:features, label_t:label,
                           bias_t: np.zeros(label.shape)})

In [202]:
test_val.shape, test_val2.shape, T_v.shape, l_curr_v.shape, whole_loss_v.shape

((15,), (15,), (300000, 15), (300000, 15), (299999, 15))

In [203]:
thr_v, bloss_v, bind_v, np.log(2)

(0.9465502307970464, 0.612421807341928, 225316, 0.6931471805599453)

In [199]:
test_val, test_val2

(array([225316, 188140, 293300, 279952, 244641, 256204,  97004, 162696,
        220095, 261495, 245996, 232931, 295490, 289861, 214622]),
 array([0.61242181, 0.61245149, 0.61245253, 0.61246121, 0.61245301,
        0.61246654, 0.6124539 , 0.61247206, 0.61245837, 0.61246974,
        0.61245133, 0.61246619, 0.61245465, 0.61246749, 0.61245388]))

In [176]:
#T_v[150000:150010,:3]

In [178]:
#cum_l_v[9999:10001,:5]

In [160]:
#l_curr_v[:20,:5]
whole_loss_v[-20:,:5]

array([[0.69149053, 0.69149053, 0.69149053, 0.69149053, 0.69149053],
       [0.69149053, 0.69149053, 0.69149053, 0.69149053, 0.69149053],
       [0.69149053, 0.69149053, 0.69149053, 0.69149053, 0.69149053],
       [0.6914905 , 0.6914905 , 0.6914905 , 0.6914905 , 0.6914905 ],
       [0.6914905 , 0.6914905 , 0.6914905 , 0.6914905 , 0.6914905 ],
       [0.6914904 , 0.6914904 , 0.6914904 , 0.6914904 , 0.6914904 ],
       [0.6914904 , 0.6914904 , 0.6914904 , 0.6914904 , 0.6914904 ],
       [0.6914904 , 0.6914904 , 0.6914904 , 0.6914904 , 0.6914904 ],
       [0.69149035, 0.69149035, 0.69149035, 0.69149035, 0.69149035],
       [0.69149035, 0.69149035, 0.69149035, 0.69149035, 0.69149035],
       [0.69149035, 0.69149035, 0.69149035, 0.69149035, 0.69149035],
       [0.6914903 , 0.6914903 , 0.6914903 , 0.6914903 , 0.6914903 ],
       [0.6914903 , 0.6914903 , 0.6914903 , 0.6914903 , 0.6914903 ],
       [0.6914903 , 0.6914903 , 0.6914903 , 0.6914903 , 0.6914903 ],
       [0.69149023, 0.69149023, 0.

In [10]:
y_v[:10, :10]

array([[0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 1., 1.],
       [0., 0., 0., 0., 1., 1., 0., 1., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 1., 0., 0., 1.],
       [0., 0., 1., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]], dtype=float32)

In [ ]:
for p in range(10000):
    for q in range(10):
        if label[axv[p, q]] != y_sub_v[p, q]:
            print('fail!!', p, q)
            break

In [ ]:
axv.shape

In [ ]:
axv[:3, :3]

In [ ]:
label[139807]

In [ ]:
features[:5, :5]